In [1]:
import rospy
import actionlib
from geometry_msgs.msg import Point
from assignment_2_2024.msg import PlanningAction, PlanningGoal  # Adjust to your pkg/action
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
from IPython.display import display
import math

In [2]:
class RobotInterface:
    def __init__(self):
        # Initialize action client
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        self.client.wait_for_server()
        
        # Robot state (updated by subscribers)
        self.robot_pos = Point()
        self.min_distance = float('inf')
        
        # Subscribers
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        rospy.Subscriber('/scan', LaserScan, self.scan_callback)

In [3]:
    def odom_callback(self, msg):
        # Update robot position (x, y)
        self.robot_pos.x = msg.pose.pose.position.x
        self.robot_pos.y = msg.pose.pose.position.y
    
    def scan_callback(self, msg):
        # Update minimum obstacle distance (ignore inf values)
        self.min_distance = min(r for r in msg.ranges if not math.isinf(r))

In [4]:
    def run(self):
        # Target input widgets
        x_input = widgets.FloatText(description="X:")
        y_input = widgets.FloatText(description="Y:")
        goal_button = widgets.Button(description="Set Goal")
        cancel_button = widgets.Button(description="Cancel Goal")
        
        # Display widgets
        display(x_input, y_input, goal_button, cancel_button)
        
        # Output widgets for feedback
        pos_label = widgets.Label(value=f"Robot Position: (0.0, 0.0)")
        dist_label = widgets.Label(value="Closest Obstacle: ∞ meters")
        display(pos_label, dist_label)
        
        # Button actions
        def set_goal(_):
            goal = PlanningGoal()
            goal.target_pose.pose.position.x = x_input.value
            goal.target_pose.pose.position.y = y_input.value
            self.client.send_goal(goal)
        
        def cancel_goal(_):
            self.client.cancel_goal()
        
        goal_button.on_click(set_goal)
        cancel_button.on_click(cancel_goal)
        
        # Update UI in real-time
        while True:
            pos_label.value = f"Robot Position: ({self.robot_pos.x:.2f}, {self.robot_pos.y:.2f})"
            dist_label.value = f"Closest Obstacle: {self.min_distance:.2f} meters"
            rospy.sleep(0.1)

In [5]:
if __name__ == "__main__":
    rospy.init_node('robot_interface')
    interface = RobotInterface()
    interface.run()

AttributeError: 'RobotInterface' object has no attribute 'odom_callback'

In [6]:
#!/usr/bin/env python

# ROS imports
import rospy
import actionlib
from geometry_msgs.msg import Point
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
from IPython.display import display
import math

# Import your action messages (make sure your package is built)
try:
    from assignment_2_2024.msg import PlanningAction, PlanningGoal
except ImportError as e:
    print(f"Error importing action messages: {e}")
    print("Did you build your package with 'catkin_make'?")
    raise

class RobotInterface:
    def __init__(self):
        # Initialize action client
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        self.client.wait_for_server()
        
        # Robot state
        self.robot_pos = Point()
        self.min_distance = float('inf')
        
        # Initialize subscribers
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.scan_sub = rospy.Subscriber('/scan', LaserScan, self.scan_callback)
    
    def odom_callback(self, msg):
        """Callback for Odometry messages"""
        self.robot_pos.x = msg.pose.pose.position.x
        self.robot_pos.y = msg.pose.pose.position.y
    
    def scan_callback(self, msg):
        """Callback for LaserScan messages"""
        valid_ranges = [r for r in msg.ranges if not math.isinf(r)]
        if valid_ranges:
            self.min_distance = min(valid_ranges)
    
    def run(self):
        """Create and display the interactive interface"""
        # Target input widgets
        x_input = widgets.FloatText(description="X:")
        y_input = widgets.FloatText(description="Y:")
        goal_button = widgets.Button(description="Set Goal")
        cancel_button = widgets.Button(description="Cancel Goal")
        
        # Display widgets
        display(x_input, y_input, goal_button, cancel_button)
        
        # Output widgets
        pos_label = widgets.Label()
        dist_label = widgets.Label()
        display(pos_label, dist_label)
        
        # Button actions
        def set_goal(_):
            goal = PlanningGoal()
            goal.target_pose.pose.position.x = x_input.value
            goal.target_pose.pose.position.y = y_input.value
            self.client.send_goal(goal)
            print(f"Goal set to: ({x_input.value}, {y_input.value})")
        
        def cancel_goal(_):
            self.client.cancel_goal()
            print("Goal cancelled")
        
        goal_button.on_click(set_goal)
        cancel_button.on_click(cancel_goal)
        
        # Update UI in real-time
        try:
            while not rospy.is_shutdown():
                pos_label.value = f"Robot Position: ({self.robot_pos.x:.2f}, {self.robot_pos.y:.2f})"
                dist_label.value = f"Closest Obstacle: {self.min_distance:.2f} meters"
                rospy.sleep(0.1)
        except KeyboardInterrupt:
            print("Interface stopped")

if __name__ == "__main__":
    try:
        rospy.init_node('robot_interface')
        interface = RobotInterface()
        print("Robot Interface Ready!")
        interface.run()
    except rospy.ROSInterruptException:
        pass

Robot Interface Ready!


FloatText(value=0.0, description='X:')

FloatText(value=0.0, description='Y:')

Button(description='Set Goal', style=ButtonStyle())

Button(description='Cancel Goal', style=ButtonStyle())

Label(value='')

Label(value='')

Interface stopped


In [15]:
#!/usr/bin/env python

# ROS Setup
import sys


import rospy
import actionlib
import ipywidgets as widgets
from IPython.display import display
import math

# ROS Messages
from geometry_msgs.msg import Point, PoseStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2024.msg import PlanningAction, PlanningGoal, PlanningResult

class RobotInterface:
    def __init__(self):
        # Action Client
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        if not self.client.wait_for_server(rospy.Duration(5.0)):
            rospy.logerr("Could not connect to action server!")
            return
        
        # Robot State
        self.robot_pos = Point()
        self.min_distance = float('inf')
        self.goal_status = "No active goal"
        
        # Subscribers
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.scan_sub = rospy.Subscriber('/scan', LaserScan, self.scan_callback)
    
    def odom_callback(self, msg):
        self.robot_pos.x = msg.pose.pose.position.x
        self.robot_pos.y = msg.pose.pose.position.y
    
    def scan_callback(self, msg):
        valid_ranges = [r for r in msg.ranges if not math.isinf(r) and r > 0.1]
        self.min_distance = min(valid_ranges) if valid_ranges else float('inf')
    
    def run(self):
        # UI Widgets
        x_input = widgets.FloatText(value=0.0, description="X:")
        y_input = widgets.FloatText(value=0.0, description="Y:")
        goal_button = widgets.Button(description="Set Goal")
        cancel_button = widgets.Button(description="Cancel Goal")
        
        status_label = widgets.Label(value="Status: Ready")
        pos_label = widgets.Label()
        dist_label = widgets.Label()
        
        display(widgets.VBox([x_input, y_input, goal_button, cancel_button, 
                            status_label, pos_label, dist_label]))
        
        def set_goal(_):
            goal = PlanningGoal()
            goal.target_pose = PoseStamped()
            goal.target_pose.pose.position.x = x_input.value
            goal.target_pose.pose.position.y = y_input.value
            self.client.send_goal(
                goal,
                done_cb=self.goal_done,
                active_cb=self.goal_active,
                feedback_cb=self.goal_feedback
            )
            status_label.value = "Status: Goal sent!"
        
        def cancel_goal(_):
            self.client.cancel_goal()
            status_label.value = "Status: Goal cancelled"
        
        goal_button.on_click(set_goal)
        cancel_button.on_click(cancel_goal)
        
        try:
            while not rospy.is_shutdown():
                pos_label.value = f"Position: ({self.robot_pos.x:.2f}, {self.robot_pos.y:.2f})"
                dist_label.value = f"Closest obstacle: {self.min_distance:.2f}m"
                rospy.sleep(0.1)
        except KeyboardInterrupt:
            pass
    
    def goal_active(self):
        self.goal_status = "Goal active"
    
    def goal_feedback(self, feedback):
        pass  # Can implement if you have feedback in your action
    
    def goal_done(self, status, result):
        if status == actionlib.GoalStatus.SUCCEEDED:
            self.goal_status = f"Success! Final position: ({result.actual_pose.position.x:.2f}, {result.actual_pose.position.y:.2f})"
        else:
            self.goal_status = f"Failed with status: {status}"

if __name__ == "__main__":
    rospy.init_node('robot_interface')
    interface = RobotInterface()
    print("Interface ready! Use the controls below.")
    interface.run()

Interface ready! Use the controls below.


[ERROR] [1746566210.885458, 2762.356000]: Could not connect to action server!


In [11]:
#!/usr/bin/env python
import rospy
import actionlib
import math
from geometry_msgs.msg import Twist, PoseStamped
from nav_msgs.msg import Odometry
from assignment_2_2024.msg import PlanningAction, PlanningFeedback, PlanningResult

class PlanningServer:
    def __init__(self):
        # Initialize ROS node if not already initialized
        try:
            rospy.init_node('planning_server', anonymous=True)
        except rospy.exceptions.ROSException:
            pass  # Already initialized
            
        self.server = actionlib.SimpleActionServer('/reaching_goal', PlanningAction, self.execute, False)
        self.cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        self.odom_sub = rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.current_pose = None
        self.server.start()
        rospy.loginfo("Action server started")

    def odom_callback(self, msg):
        self.current_pose = msg.pose.pose

    def execute(self, goal):
        rospy.loginfo(f"New goal received: ({goal.target_pose.pose.position.x}, {goal.target_pose.pose.position.y})")
        rate = rospy.Rate(10)
        success = True
        twist = Twist()
        
        while not rospy.is_shutdown():
            if self.server.is_preempt_requested():
                rospy.loginfo("Goal preempted")
                self.server.set_preempted()
                success = False
                break

            if self.current_pose is None:
                rospy.logwarn("Waiting for odometry data...")
                rate.sleep()
                continue

            # Calculate distance to goal
            dx = goal.target_pose.pose.position.x - self.current_pose.position.x
            dy = goal.target_pose.pose.position.y - self.current_pose.position.y
            distance = math.sqrt(dx**2 + dy**2)

            # Stop if reached goal
            if distance < 0.1:  # Threshold
                twist.linear.x = 0
                twist.angular.z = 0
                self.cmd_vel_pub.publish(twist)
                rospy.loginfo("Goal reached!")
                break

            # Calculate angle to goal
            target_angle = math.atan2(dy, dx)
            current_angle = 2 * math.atan2(
                self.current_pose.orientation.z,
                self.current_pose.orientation.w
            )
            angle_error = target_angle - current_angle
            angle_error = math.atan2(math.sin(angle_error), math.cos(angle_error))

            # Proportional control
            twist.linear.x = min(0.5, distance * 0.5)
            twist.angular.z = angle_error * 1.0

            self.cmd_vel_pub.publish(twist)
            rate.sleep()

        if success:
            result = PlanningResult()
            result.actual_pose = self.current_pose
            result.stat = "Goal reached successfully"
            self.server.set_succeeded(result)
            rospy.loginfo("Goal succeeded")

if __name__ == "__main__":
    try:
        server = PlanningServer()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass

[INFO] [1746565402.358821, 2762.356000]: Action server started


In [13]:
import rospy
import actionlib
import ipywidgets as widgets
from IPython.display import display
from geometry_msgs.msg import PoseStamped
from assignment_2_2024.msg import PlanningAction, PlanningGoal
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan

# Initialize ROS node
try:
    rospy.init_node('robot_interface', anonymous=True)
except:
    pass

class RobotInterface:
    def __init__(self):
        # Action client
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        if not self.client.wait_for_server(timeout=rospy.Duration(5)):
            rospy.logwarn("Couldn't connect to action server!")
        
        # Robot state
        self.position = (0, 0)
        self.min_distance = float('inf')
        
        # Subscribers
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        rospy.Subscriber('/scan', LaserScan, self.scan_callback)
        
        # Create UI
        self.create_ui()
    
    def odom_callback(self, msg):
        self.position = (msg.pose.pose.position.x, 
                        msg.pose.pose.position.y)
    
    def scan_callback(self, msg):
        valid_ranges = [r for r in msg.ranges if r > 0.1 and not math.isinf(r)]
        self.min_distance = min(valid_ranges) if valid_ranges else float('inf')
    
    def create_ui(self):
        # Input widgets
        self.x_input = widgets.FloatText(value=0.0, description='X:')
        self.y_input = widgets.FloatText(value=0.0, description='Y:')
        
        # Buttons
        set_goal_btn = widgets.Button(description='Set Goal')
        cancel_goal_btn = widgets.Button(description='Cancel Goal')
        
        # Display widgets
        position_label = widgets.Label()
        distance_label = widgets.Label()
        status_output = widgets.Output()
        
        # Layout
        display(widgets.VBox([
            widgets.HBox([self.x_input, self.y_input]),
            widgets.HBox([set_goal_btn, cancel_goal_btn]),
            position_label,
            distance_label,
            status_output
        ]))
        
        # Button actions
        def on_set_goal(b):
            goal = PlanningGoal()
            goal.target_pose = PoseStamped()
            goal.target_pose.pose.position.x = self.x_input.value
            goal.target_pose.pose.position.y = self.y_input.value
            
            with status_output:
                print(f"Setting goal to: ({self.x_input.value}, {self.y_input.value})")
            self.client.send_goal(goal)
        
        def on_cancel_goal(b):
            with status_output:
                print("Canceling current goal")
            self.client.cancel_goal()
        
        set_goal_btn.on_click(on_set_goal)
        cancel_goal_btn.on_click(on_cancel_goal)
        
        # Update display
        while True:
            position_label.value = f"Position: ({self.position[0]:.2f}, {self.position[1]:.2f})"
            distance_label.value = f"Closest obstacle: {self.min_distance:.2f}m"
            rospy.sleep(0.1)

# Start interface
interface = RobotInterface()

[WARN] [1746565657.325186, 2762.356000]: Couldn't connect to action server!


ROSInterruptException: ROS shutdown request

In [14]:
#!/usr/bin/env python
import rospy
import actionlib
import ipywidgets as widgets
from IPython.display import display
from geometry_msgs.msg import PoseStamped, Point
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2024.msg import PlanningAction, PlanningGoal
import math

class RobotInterface:
    def __init__(self):
        # Initialize ROS node (only once)
        try:
            rospy.init_node('robot_interface', anonymous=True)
        except rospy.ROSException:
            pass  # Already initialized
        
        # Connect to existing action server (bug_as.py)
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        if not self.client.wait_for_server(timeout=rospy.Duration(5)):
            rospy.logerr("Couldn't connect to action server! Is bug_as.py running?")
            return
        
        # Robot state
        self.position = Point()
        self.min_distance = float('inf')
        
        # Subscribers
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        rospy.Subscriber('/scan', LaserScan, self.scan_callback)
        
        # Create UI
        self.create_widgets()
    
    def odom_callback(self, msg):
        self.position = msg.pose.pose.position
    
    def scan_callback(self, msg):
        valid_ranges = [r for r in msg.ranges if not math.isinf(r) and r > 0.1]
        self.min_distance = min(valid_ranges) if valid_ranges else float('inf')
    
    def create_widgets(self):
        # Input widgets
        self.x_input = widgets.FloatText(description="X:")
        self.y_input = widgets.FloatText(description="Y:")
        
        # Control buttons
        set_goal_btn = widgets.Button(description="Set Goal")
        cancel_goal_btn = widgets.Button(description="Cancel Goal")
        
        # Status display
        self.position_label = widgets.Label()
        self.distance_label = widgets.Label()
        self.status_output = widgets.Output()
        
        # Layout
        display(widgets.VBox([
            widgets.HBox([self.x_input, self.y_input]),
            widgets.HBox([set_goal_btn, cancel_goal_btn]),
            self.position_label,
            self.distance_label,
            self.status_output
        ]))
        
        # Button actions
        set_goal_btn.on_click(self.set_goal)
        cancel_goal_btn.on_click(self.cancel_goal)
        
        # Start UI update loop
        self.update_ui()
    
    def set_goal(self, _):
        goal = PlanningGoal()
        goal.target_pose = PoseStamped()
        goal.target_pose.pose.position.x = self.x_input.value
        goal.target_pose.pose.position.y = self.y_input.value
        
        with self.status_output:
            print(f"Goal set to: ({self.x_input.value}, {self.y_input.value})")
        self.client.send_goal(goal)
    
    def cancel_goal(self, _):
        with self.status_output:
            print("Goal cancelled")
        self.client.cancel_goal()
    
    def update_ui(self):
        while not rospy.is_shutdown():
            self.position_label.value = f"Position: ({self.position.x:.2f}, {self.position.y:.2f})"
            self.distance_label.value = f"Closest obstacle: {self.min_distance:.2f}m"
            rospy.sleep(0.1)

# Start interface
if __name__ == "__main__":
    interface = RobotInterface()
    rospy.spin()

[ERROR] [1746565693.943070, 2762.356000]: Couldn't connect to action server! Is bug_as.py running?
